In [ ]:
import airsim
import numpy as np
import time
import torch
from yolov7.models.experimental import attempt_load
from yolov7.utils.general import non_max_suppression, scale_coords
from yolov7.utils.datasets import letterbox
import cv2

client = airsim.MultirotorClient()
client.confirmConnection()
client.enableApiControl(True)
client.armDisarm(True)

client.takeoffAsync().join()

model = attempt_load('yolov7.pt', map_location='cpu')  #YOLOv7 weights here
model.eval()

def preprocess_image(img):
    img = letterbox(img, 640, stride=32, auto=True)[0]
    img = img.transpose((2, 0, 1))
    img = np.ascontiguousarray(img)
    img = torch.from_numpy(img).float()
    img /= 255.0
    if img.ndimension() == 3:
        img = img.unsqueeze(0)
    return img

def detect_tanks(img):
    img = preprocess_image(img)
    pred = model(img)[0]
    pred = non_max_suppression(pred, 0.25, 0.45, classes=[0], agnostic=True)  # Assuming '0' is the class for tanks
    return pred

def get_drone_state():
    state = client.getMultirotorState().kinematics_estimated
    position = state.position
    return position

# move drone towards a target
def move_to_target(target_coords, speed=5):
    while True:
        position = get_drone_state()
        direction = np.array([target_coords[0] - position.x_val,
                              target_coords[1] - position.y_val,
                              target_coords[2] - position.z_val])
        distance = np.linalg.norm(direction)

        if distance < 1:
            client.hoverAsync().join()
            break

        client.moveByVelocityAsync(direction[0], direction[1], direction[2], speed).join()

# Function to track and follow the tank
def track_target():
    while True:
        # Capture front-center camera image
        response = client.simGetImage("front_center", airsim.ImageType.Scene)
        if response is None:
            continue
        img = np.frombuffer(response, dtype=np.uint8).reshape(480, 640, 3)  # Assume 480x640 resolution

        detections = detect_tanks(img)

        if detections[0] is not None:
            for det in detections[0]:
                xywh = det[0:4].cpu().numpy()
                x_center, y_center = xywh[0], xywh[1]
                width, height = xywh[2], xywh[3]

                target_coords = [x_center, y_center, -10]  # target altitude set to 10 meters

                move_to_target(target_coords)

        time.sleep(0.5)

target_coords = [10, 10, -10]
move_to_target(target_coords)

track_target()

client.landAsync().join()
client.armDisarm(False)
client.enableApiControl(False)
